In [26]:
from catboost import CatBoostRegressor
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials


N_HYPEROPT_PROBES = 60
HYPEROPT_ALGO = tpe.suggest

In [27]:
df_train = pd.read_csv('./data/train_filtrado.csv')
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)
df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.01)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)


FileNotFoundError: [Errno 2] File b'./data/train_filtrado.csv' does not exist: b'./data/train_filtrado.csv'

In [28]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [29]:
clust_train = pd.read_csv('./data/clustering_train.csv')
clust_test= pd.read_csv('./data/clustering_test.csv')

sim_train = pd.read_csv('./data/train_similares.csv')
sim_test= pd.read_csv('./data/test_similares.csv')

df_train_f = pd.merge(df_train_f, clust_train, on= 'id', how= 'left').rename(columns = {'label': 'label_clustering'})
df_test_f = pd.merge(df_test_f, clust_test, on= 'id', how= 'left').rename(columns = {'label': 'label_clustering'})
df_train_f = pd.merge(df_train_f, sim_train, on= 'id', how= 'left').rename(columns = {'similares_count_y': 'label_similares'})
df_test_f = pd.merge(df_test_f,sim_test , on= 'id', how= 'left').rename(columns = {'similares_count_y': 'label_similares'})

FileNotFoundError: [Errno 2] File b'./data/clustering_train.csv' does not exist: b'./data/clustering_train.csv'

In [ ]:
df_train_f = features.features_intervalos(df_train_f)
df_test_f = features.features_intervalos(df_test_f)

In [ ]:
# Paso a integer las features

def feature_convertion(df,features, multiplicar):
    df = df.copy()
    
    for feature in features:
        if multiplicar:
            df[feature] = df[feature].map(lambda x: x*1000)
        df[feature] = df[feature].astype('int64')
        
    return df


features_int = ['ditancia_centro_bins_unif','ditancia_centro_bins_perc','distancia_centro_mexico_bins_unif',
            'distancia_centro_mexico_bins_perc','distancia_ciudad_cara_bins_unif',
            'distancia_ciudad_cara_bins_perc','promedio_id_zona_bins_unif', 'promedio_id_zona_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc', 
            'promedio_precio_tipo_propiedad_ciudad_bins_unif','promedio_precio_tipo_propiedad_ciudad_bins_perc',
            'antiguedad_bins_unif', 'antiguedad_bins_perc', 'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'tam_ambientes_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'habitaciones', 'banos',
            'metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad','garages', 'gimnasio',
            'usosmultiples', 'piscina', 'gimnasio']

df_train_f = feature_convertion(df_train_f, features_int, False)

In [ ]:
features_mult = ['promedio_id_zona', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
                'tam_ambientes', 'promedio_precio_tipo_propiedad_ciudad', 'distancia_centro_mexico',
                 'distancia_ciudad_centrica', 'distancia_ciudad_cara', 'varianza_precio_ciudad', 
                 'promedio_precio_habitaciones_banos_garages','promedio_precio_tipo_propiedad_ciudad']


df_train_f = feature_convertion(df_train_f, features_mult, True)

df_train_f['lat'] = df_train_f['lat'].map(lambda x: x*1000000)
df_train_f['lng'] = df_train_f['lng'].map(lambda x: x*1000000)
df_train_f['lat'] = df_train_f['lng'].astype('int64')
df_train_f['lng'] = df_train_f['lng'].astype('int64')

### Busqueda de Hiperparametros

In [ ]:

features =['habitaciones', 'banos','metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad',
           'garages', 'gimnasio', 'usosmultiples', 'piscina', 'gimnasio',
           'promedio_id_zona', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
            'tam_ambientes', 'promedio_precio_tipo_propiedad_ciudad', 'distancia_centro_mexico',
            'distancia_ciudad_centrica', 'distancia_ciudad_cara', 'varianza_precio_ciudad', 
            'promedio_precio_habitaciones_banos_garages','promedio_precio_tipo_propiedad_ciudad']



x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


D_train = catboost.Pool(x_train, y_train)
D_test = catboost.Pool(x_test, y_test)

def get_catboost_params(space):
    params = dict()
    params['learning_rate'] = space['learning_rate']
    params['depth'] = int(space['depth'])
    params['l2_leaf_reg'] = space['l2_leaf_reg']
    return params


obj_call_count = 0
cur_best_loss = np.inf
def objective(space):
    
    params = get_catboost_params(space)

    model = catboost.CatBoostClassifier(iterations=200,
                                        learning_rate=params['learning_rate'],
                                        depth=int(params['depth']),
                                        eval_metric='MAE',
                                        l2_leaf_reg=params['l2_leaf_reg'],
                                        early_stopping_rounds=3000,
                                        od_type="Iter",
                                        verbose= False
                                        )
    
    model.fit(D_train, eval_set=D_test, verbose= False)
    
    y_pred = model.predict_proba(D_test.get_features())
    
    return utils.MAE(df_test_f['precio'].values, y_pred)

In [ ]:
#  space = {
#          'depth': hp.quniform("depth", 1, 6, 10),
#          'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
#         'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
#         }


# best = hyperopt.fmin(fn=objective,
#                      space=space,
#                      algo=tpe.suggest,
#                      max_evals=200,
#                     verbose=1)

# print('-'*50)
# print('The best params:')
# print( best )

In [ ]:
features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales',  'provincia', 'tipodepropiedad',
            'gimnasio', 'usosmultiples', 'piscina', 'fecha']


features_test = ['tam_ambientes','promedio_precio_tipo_propiedad_ciudad',
                 'count_tipo_propiedad_ciudad','lat', 'lng',
                    'varianza_precio_ciudad','puntaje',
                 'distancia_ciudad_cara', 'distancia_ciudad_centrica' ,
                 'distancia_ciudad_cara','promedio_id_zona',
                 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'promedio_precio_habitaciones_banos_garages',
                'ditancia_centro_bins_unif','ditancia_centro_bins_perc','distancia_centro_mexico_bins_unif',
            'distancia_centro_mexico_bins_perc','distancia_ciudad_cara_bins_unif',
            'distancia_ciudad_cara_bins_perc','promedio_id_zona_bins_unif', 'promedio_id_zona_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc', 
            'promedio_precio_tipo_propiedad_ciudad_bins_unif','promedio_precio_tipo_propiedad_ciudad_bins_perc',
            'antiguedad_bins_unif', 'antiguedad_bins_perc', 'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'tam_ambientes_bins_perc',
            'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif']
    
features += features_test

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


# train_labels = 'precio'
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=300,
                             learning_rate=0.05,
                             depth=8,
                             eval_metric='MAE',
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 1,
                             od_wait=100,
                             l2_leaf_reg= 3,
                             random_seed = None)
# Fit model
model.fit(x_train, y_train, cat_features=features, eval_set=(x_test, y_test),plot=True)

In [ ]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)

cat_mae_train = utils.MAE(y_train, y_pred_train)
cat_mae_test = utils.MAE(y_test, y_pred_test)

print(f"MAE LightGBM (train): {cat_mae_train:.5f}")
print(f"MAE LightGBM (test): {cat_mae_test:.5f}")